In [6]:
%load_ext autoreload
%autoreload 2

import os 
# os.environ['R_HOME']= r'C:\Users\tomha\miniconda3\envs\octagon_analysis\lib\R'
os.environ['R_HOME']= r'D:\Users\Tom\miniconda3\envs\octagon_analysis\lib\R'
# os.environ['R_HOME']= '/home/tom/miniconda3/envs/octagon_analysis/lib/R'

import rpy2

import rpy2.robjects as robjects
print(robjects.r('R.version.string'))

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import globals
import data_strings
import data_extraction.get_indices as get_indices
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
from trajectory_analysis import trajectory_vectors
from plotting import plot_octagon
import parse_data.identify_filepaths as identify_filepaths 
from data_extraction.trial_list_filters import filter_trials_other_visible
from analysis import opponent_visibility
from ipywidgets import IntProgress
from IPython.display import display
import time
from pymer4.models import Lmer
import populate_dataframes



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1] "R version 4.1.3 (2022-03-10)"



In [31]:
k = 20
split_dataframes = True

### load data

In [32]:
import pickle

analysis_dir = os.path.join('..', 'data')
analysis_file = 'analysis_results_2levelsforFirstSeenWall.pkl'
filename = os.path.join(analysis_dir, analysis_file)
# load the analysis results
with open(filename, 'rb') as f:
    analysis_results = pickle.load(f)

### populate dataframes for glm input

In [33]:
# populate dataframes for solo, solosocial, and social analysis_type
glm_df_solo = populate_dataframes.populate_dataframe(analysis_results, analysis_type='solo')
glm_df_solosocial = populate_dataframes.populate_dataframe(analysis_results, analysis_type='solosocial')
glm_df_social = populate_dataframes.populate_dataframe(analysis_results, analysis_type='social')


### create reference to dataframes

In [34]:
dataframes = {
    'glm_df_solo': glm_df_solo,
    'glm_df_solosocial': glm_df_solosocial,
    'glm_df_social': glm_df_social
}

### shuffle the dataframes for k-fold index selection


In [35]:
shuffled_dataframes = os.path.join('..', 'data', 'shuffled_dataframes.pkl')

if split_dataframes:
    
    # shuffle each dataframe
    for name, df in dataframes.items():
        dataframes[name] = df.sample(frac=1).reset_index(drop=True)

    # pickle save shuffled dataframes to sandbox > data, as one dictionary
    with open(shuffled_dataframes, 'wb') as f:
        pickle.dump(dataframes, f)

else:
    # load the shuffled dataframes
    with open(shuffled_dataframes, 'rb') as f:
        shuffled_dataframes = pickle.load(f)



### Split each dataframe into folds

In [ ]:
# split each dataframe into k equal parts
split_dataframes = {name: np.array_split(df, k) for name, df in dataframes.items()}

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
def fit_lmer_models(dataframes, split_dataframes, model_formula):
    """
    For each dataframe type, hold out one fold of the split dataframe, 
    and fit a Lmer binomial model on the remaining folds.

    Parameters:
    - dataframes: dict, original dataframes
    - split_dataframes: dict, split dataframes into folds
    - model_formula: str, formula for the Lmer model

    Returns:
    - models_dict: dict, models for each dataframe type
    """
    models_dict = {}

    for df_name, folds in split_dataframes.items():
        models = []
        print(f"Fitting models for {df_name}...")
        for i in range(len(folds)):
            # Combine all folds except the i-th fold
            train_data = pd.concat(folds[:i] + folds[i+1:], ignore_index=True)
            
            # Fit the Lmer model
            model = Lmer(model_formula, data=train_data, family='binomial')
            model.fit()
            models.append(model)
        
        models_dict[df_name] = models
    
    return models_dict

In [ ]:
from contextlib import redirect_stdout

def fit_models(analysis_type, split_dataframes_dict, model_formula, k):
    
    # get the dataframe for the analysis type
    dataframe_name = f'glm_df_{analysis_type}'
    split_dataframes = split_dataframes_dict[dataframe_name]

    models = []
    max_count = k
    f = IntProgress(min=0, max=max_count, description='Fitting models')
    display(f)

    # Suppress the output of the models fitting process
    with open(os.devnull, 'w') as fnull:
        with redirect_stdout(fnull):
            for i, df in enumerate(split_dataframes):
                model = Lmer(model_formula, data=df, family='binomial')
                model.fit()
                models.append(model)
                print(f"Model {i} fit with {len(df)} rows")
                f.value += 1


    
    return models


def calculate_predictions(models, original_df_size, dfs_with_removed_row_sampled, random_indices):
    
    predictions = np.full(len(dfs_with_removed_row_sampled), np.nan)
    predictions_maintained_index = np.full(original_df_size, np.nan)
    for i, model in enumerate(models):
        # get the row that was removed for this model
        removed_row = dfs_with_removed_row_sampled[i]
        
        # get the prediction for this row
        prediction = model.predict(removed_row, skip_data_checks=True, verify_predictions=False)
        
        # assign the prediction to the correct index in the predictions array
        predictions_maintained_index[random_indices[i]] = prediction[0]

        # also assign the prediction to the next index of a new array
        predictions[i] = prediction[0]

    return predictions, predictions_maintained_index

def calculate_likelihoods(df, predictions_maintained_index, random_indices):
    
    # calculate the metric for each prediction
    likelihoods = np.full(len(random_indices), np.nan)
    for i, idx in enumerate(random_indices):
        predicted_output = predictions_maintained_index[idx]
        true_output = df.iloc[idx]['ChooseHigh']
        likelihood = predicted_output**true_output * (1 - predicted_output)**(1 - true_output)
        likelihoods[i] = likelihood

    return likelihoods

def calculate_nll(likelihoods):
    # #### sum the logs of the likelihoods, and take the negative
    summed_log_likelihoods = np.sum(np.log(likelihoods)) 
    nll = -summed_log_likelihoods

    return nll

def save_cross_validation_results(name, model_formula, df, random_indices, predictions, nll):
    ''' Save the cross-validation results to a file. '''
    
    cross_validation_results = {
        "name": name,
        "model_formula": model_formula,
        "dataframe": df,
        "random_indices" : random_indices,
        # "models" : models,
        "predictions" : predictions,
        "nll" : nll
    }

   # Save the cross-validation results to a file
    dir = os.path.join('..', 'data')
    filename = f'CV_results_{name}.pickle'
    filepath = os.path.join(dir, filename)
    with open(filepath, 'wb') as f:
        pickle.dump(cross_validation_results, f)

    print("CV data saved to: ", filepath)